In [1]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn
import pickle

wandb.login()

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


True

In [2]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=-1
DIMENSIONS=200

N_LABELS=2
MAX_LEN = 256
EPOCHS=100
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=16
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [3]:
'''Preparing dataset'''

file_path = 'datasets/subtaskA_glove_train_dev_monolingual.pkl'

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Extract the individual datasets from the loaded data
train_x = data['train_x']
train_y = data['train_y']
dev_x = data['dev_x']
dev_y = data['dev_y']

if SAMPLES_TO_TRAIN!=-1:
    random_indices = np.random.choice(train_x.shape[0], size=SAMPLES_TO_TRAIN, replace=False)
    train_x = train_x[random_indices]
    train_y = train_y[random_indices]

    random_indices_dev = np.random.choice(dev_x.shape[0], size=int(train_x.shape[0] * 0.2), replace=False)
    dev_x = dev_x[random_indices_dev]
    dev_y = pd.Series(dev_y.to_numpy()[random_indices_dev])

print(train_x.shape)
print(train_y.shape)
print(dev_x.shape)
print(dev_y.shape)

print(type(train_y))
print(type(dev_y))

(119757, 200)
(119757,)
(5000, 200)
(5000,)
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [4]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y.values, test_size=0.2, random_state=42)
X_dev=dev_x
y_dev=dev_y.values

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Compute Metrics
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

metrics = {
    'accuracy': acc,
    'f1': f1,
    'auc': auc,
    'precision': precision,
    'recall': recall,
}

print(metrics)

{'accuracy': 0.7407792207792208, 'f1': 0.7402394586153046, 'auc': 0.7410130401354551, 'precision': 0.7277379733879222, 'recall': 0.753177966101695}


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a base model
rf = RandomForestClassifier(random_state=42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='f1')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters
print(grid_search.best_params_)

# Train and predict using the best model
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)


Fitting 3 folds for each of 144 candidates, totalling 432 fits
{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score

# Train Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42, min_samples_split=10,min_samples_leaf=2,max_depth=None)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Compute Metrics
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

metrics = {
    'accuracy': acc,
    'f1': f1,
    'auc': auc,
    'precision': precision,
    'recall': recall,
}

print(metrics)

{'accuracy': 0.8009769539078156, 'f1': 0.7819603896994923, 'auc': 0.7986958061216461, 'precision': 0.8213702315749015, 'recall': 0.7461592178770949}


In [7]:
with open('SavedModels/'+'randomforest_'+str(round(SAMPLES_TO_TRAIN/1000))+'k.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)

In [1]:
# All samples trained with training dataset
print(''''accuracy': 0.8009769539078156, 'f1': 0.7819603896994923, 'auc': 0.7986958061216461, 'precision': 0.8213702315749015, 'recall': 0.7461592178770949''')

'accuracy': 0.8009769539078156, 'f1': 0.7819603896994923, 'auc': 0.7986958061216461, 'precision': 0.8213702315749015, 'recall': 0.7461592178770949
